In [1]:
#Import packages needed
import os
import subprocess

cwd = os.getcwd()

# Clone the repository
repository_url = "https://github.com/Xpehutta/llm4sql.git"
destination_folder = cwd + "/llm4sql"

# Ensure the destination folder does not already exist
if os.path.exists(destination_folder):
    print(f"Destination folder {destination_folder} already exists.")
else:
    try:
        # Run the git clone command
        subprocess.run(["git", "clone", repository_url, destination_folder], check=True)
        print(f"Repository cloned successfully to {destination_folder}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while cloning the repository: {e}")

# Define the source and destination directories
source_dir = cwd +'/llm4sql/data-model-generator'
destination_dir = cwd +'/llm4sql/data_model_generator'

# Step 1: Verify that the source directory exists
if not os.path.exists(source_dir):
    print(f"Source directory {source_dir} does not exist.")
    exit(1)

# Step 2: Rename the directory using os.rename
try:
    os.rename(source_dir, destination_dir)
    print(f"Successfully renamed {source_dir} to {destination_dir}")
except OSError as e:
    print(f"An error occurred while renaming the directory: {e}")

# Optional: Verify that the destination directory exists
if os.path.exists(destination_dir):
    print(f"Destination directory {destination_dir} exists.")
else:
    print(f"Destination directory {destination_dir} does not exist.")

# Define the source and destination directories
destination_dir = destination_folder + '/data_model_generator'

# Change the current working directory to the requered one
try:
    os.chdir(destination_dir)
    print(f"Changed directory to {os.getcwd()}")
except OSError as e:
    print(f"An error occurred while changing directory: {e}")
    exit(1)

# Step 4: Install the package using pip with the requirements.txt file
requirements_file = 'requirements.txt'
if os.path.exists(requirements_file):
    try:
        subprocess.run(['pip', 'install', '-q', '-r', requirements_file], check=True)
        print("Dependencies installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while installing dependencies: {e}")
else:
    print(f"No {requirements_file} found. Skipping installation of dependencies.")

# Return to current working directory 
try:
    os.chdir(cwd)
    print(f"Changed directory to {os.getcwd()}")
except OSError as e:
    print(f"An error occurred while changing directory: {e}")
    exit(1)

Repository cloned successfully to /kaggle/working/llm4sql
Successfully renamed /kaggle/working/llm4sql/data-model-generator to /kaggle/working/llm4sql/data_model_generator
Destination directory /kaggle/working/llm4sql/data_model_generator exists.
Changed directory to /kaggle/working/llm4sql/data_model_generator
Dependencies installed successfully.
Changed directory to /kaggle/working


In [2]:
import sqlglot
import sqlglot.expressions as exp
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict

from faker import Faker

import pandas as pd
import numpy as np
import random
import time

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW

from peft import LoraConfig, get_peft_model, TaskType

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from joblib import Parallel, delayed

from llm4sql.data_model_generator.src.data_model_generator import *
from llm4sql.data_model_generator.src.sql_l_rouge import *
from llm4sql.data_model_generator.src.sql_similarity import *
from llm4sql.data_model_generator.model.create_data_model_excel import *

In [3]:
data_model = DataModelExcel()
data_model.create_data_model()

Data model created successfully: data_model.xlsx


In [4]:
generator = DataModelGenerator()
generator.load_data_model()
dataset, queries = generator.generate_dataset(num_queries=5000)

In [5]:
sql_queries = list(queries['queries'])

# Example usage
similarity = SQLSimilarity()
query1 = sql_queries[0]
query2 = sql_queries[4999]

print(query1, '\n')
print(query2)

# Normalize queries
norm1 = similarity.normalize_query(query1)
norm2 = similarity.normalize_query(query2)

# Calculate similarities
if norm1 and norm2:
    ast_sim = similarity.ast_similarity(norm1, norm2)
    component_sim = similarity.component_similarity(norm1, norm2)
    print(f"\nAST Similarity: {ast_sim:.2f}")
    print("Component Similarities:")
    for k, v in component_sim.items():
        print(f"  {k}: {v:.2f}")
    
    # Combined similarity score (custom weights)
    combined_score = (
        0.4 * ast_sim + 
        0.3 * component_sim.get("tables", 0) +
        0.2 * component_sim.get("columns", 0) +
        0.1 * component_sim.get("conditions", 0)
    )
    print(f"\nCombined Similarity Score: {combined_score:.2f}")
else:
    print("\nError in query normalization")

SELECT *
FROM followers
 JOIN customers ON followers.user_id = customers.customer_id
WHERE customers.customer_id = 760 AND customers.name = True; 

SELECT activity_id, activity_type
FROM activity_logs
WHERE activity_logs.activity_id = 371 AND activity_logs.activity_type = False;

AST Similarity: 0.61
Component Similarities:
  conditions: 0.00
  tables: 0.00
  columns: 0.00

Combined Similarity Score: 0.25


In [6]:
N = 100
# Precompute normalized queries for all SQL queries
normalized_queries = [similarity.normalize_query(q) for q in sql_queries[:N]]

# Helper function to compute pairwise similarities for a single query
def compute_row(norm1, normalized_queries):
    ast_scores = [similarity.ast_similarity(norm1, norm2) for norm2 in normalized_queries]
    tables_scores = [similarity.component_similarity(norm1, norm2).get("tables", 0) for norm2 in normalized_queries]
    columns_scores = [similarity.component_similarity(norm1, norm2).get("columns", 0) for norm2 in normalized_queries]
    conditions_scores = [similarity.component_similarity(norm1, norm2).get("conditions", 0) for norm2 in normalized_queries]
    return ast_scores, tables_scores, columns_scores, conditions_scores

# Parallel computation of rows
results = Parallel(n_jobs=-1)(delayed(compute_row)(norm1, normalized_queries) 
                              for norm1 in normalized_queries)

# Unpack results into separate matrices
ast_matrix = np.array([row[0] for row in results])
tables_matrix = np.array([row[1] for row in results])
columns_matrix = np.array([row[2] for row in results])
conditions_matrix = np.array([row[3] for row in results])

# Combine matrices using weighted sum
combined_matrix = (
    0.4 * ast_matrix +
    0.3 * tables_matrix +
    0.2 * columns_matrix +
    0.1 * conditions_matrix
)

# Convert the result to a DataFrame
similarity_df = pd.DataFrame(combined_matrix, index=sql_queries[:N], columns=sql_queries[:N])

In [7]:
pd.DataFrame(similarity_df.iloc[0]).sort_values(by = [similarity_df.columns[0]], ascending=False).head()

,SELECT *\nFROM followers\n JOIN customers ON followers.user_id = customers.customer_id\nWHERE customers.customer_id = 760 AND customers.name = True;
SELECT *\nFROM followers\n JOIN customers ON followers.user_id = customers.customer_id\nWHERE customers.customer_id = 760 AND customers.name = True;,1.000000
SELECT *\nFROM followers\nLEFT JOIN customers ON followers.user_id = customers.customer_id\nWHERE customers.name = False AND customers.customer_id = 80;,0.865714
SELECT *\nFROM orders\nLEFT JOIN customers ON orders.customer_id = customers.customer_id\nWHERE orders.customer_id = 183 AND customers.name = True;,0.633333
SELECT *\nFROM feedbacks\n JOIN customers ON feedbacks.customer_id = customers.customer_id\nWHERE feedbacks.customer_id > 347 AND customers.name = True;,0.629524
SELECT *\nFROM messages\n JOIN customers ON messages.sender_id = customers.customer_id\nWHERE messages.sender_id = 952 AND customers.name = False;,0.600000


In [8]:
torch.cuda.empty_cache()

In [9]:
#from datasets import load_metric
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,  # Rank of the LoRA matrices
    lora_alpha=32,  # Scaling factor for LoRA weights
    lora_dropout=0.1  # Dropout for LoRA layers
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

class SQLDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input_text = item["input"]
        output_text = item["output"]

        full_text = f"{input_text} -> {output_text}"
        tokenized = self.tokenizer(full_text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": tokenized["input_ids"].squeeze(),
            "attention_mask": tokenized["attention_mask"].squeeze(),
            "labels": tokenized["input_ids"].squeeze()
        }


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

trainable params: 786,432 || all params: 1,347,258,368 || trainable%: 0.0584


In [10]:
# Assuming `dataset` is your original dataset (list of dictionaries or similar structure)
# Split the dataset into training and validation sets
train_dataset, val_dataset = train_test_split(dataset[['input', 'output']], test_size=0.1, random_state=42, stratify = dataset['group'])

# Create DataLoaders for training and validation
sql_dataset_train = SQLDataset(train_dataset, tokenizer, max_length=128)
sql_dataset_val = SQLDataset(val_dataset, tokenizer, max_length=128)

train_dataloader = DataLoader(sql_dataset_train, batch_size=4, shuffle=True)
val_dataloader = DataLoader(sql_dataset_val, batch_size=4, shuffle=False)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, no_deprecation_warning=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

Using device: cuda


In [12]:
# Initialize the SQLRougeL class
sql_rouge = SQLRougeL()


# Start overall timer
overall_start_time = time.time()

for epoch in range(3):  # Number of epochs
    epoch_start_time = time.time()
    
    # Training phase
    model.train()
    total_train_loss = 0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")
    
    # End of training phase timer
    train_end_time = time.time()
    train_duration = train_end_time - epoch_start_time
    print(f"Epoch {epoch + 1}, Training Duration: {train_duration:.2f} seconds")
    
    # Validation phase
    model.eval()
    total_val_loss = 0
    total_rouge_l_f1 = 0
    total_queries = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_val_loss += loss.item()
            # Generate predictions
            generated_tokens = model.generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_length=256,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False  # Deterministic generation for evaluation
            )
            generated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            reference_text = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
            for gen_query, ref_query in zip(generated_text, reference_text):
                total_queries += 1
                # Compute ROUGE-L-SQL
                metrics = sql_rouge.rouge_l_sql(gen_query, ref_query)
                total_rouge_l_f1 += metrics['f1_score']
    avg_val_loss = total_val_loss / len(val_dataloader)
    avg_rouge_l_f1 = total_rouge_l_f1 / total_queries if total_queries > 0 else 0
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}")
    print(f"Epoch {epoch + 1}, ROUGE-L-SQL F1 Score: {avg_rouge_l_f1:.4f}")
    
    # End of epoch timer
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch + 1}, Total Epoch Duration: {epoch_duration:.2f} seconds")

# End overall timer
overall_end_time = time.time()
overall_duration = overall_end_time - overall_start_time
print(f"Overall Training Duration: {overall_duration:.2f} seconds")

Epoch 1, Training Loss: 0.1687
Epoch 1, Training Duration: 954.77 seconds
Epoch 1, Validation Loss: 0.1677
Epoch 1, ROUGE-L-SQL F1 Score: 0.4067
Epoch 1, Total Epoch Duration: 1723.02 seconds
Epoch 2, Training Loss: 0.1655
Epoch 2, Training Duration: 953.36 seconds
Epoch 2, Validation Loss: 0.1664
Epoch 2, ROUGE-L-SQL F1 Score: 0.4392
Epoch 2, Total Epoch Duration: 1721.41 seconds
Epoch 3, Training Loss: 0.1640
Epoch 3, Training Duration: 952.15 seconds
Epoch 3, Validation Loss: 0.1664
Epoch 3, ROUGE-L-SQL F1 Score: 0.4497
Epoch 3, Total Epoch Duration: 1720.05 seconds
Overall Training Duration: 5164.47 seconds


In [15]:
# Save the LoRA adapter weights
model.save_pretrained("./lora_sql_model")
tokenizer.save_pretrained("./lora_sql_model")

('./lora_sql_model/tokenizer_config.json',
 './lora_sql_model/special_tokens_map.json',
 './lora_sql_model/tokenizer.json')

In [18]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define input text (example: partial SQL query)
test_input = "SELECT * FROM products JOIN "

# Tokenize the input
input_ids = tokenizer(test_input, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

# Generate output using the model's generate method
with torch.no_grad():  # Disable gradient computation during inference
    generated_tokens = model.generate(
        **input_ids,
        max_length=256,  # Allow sufficient length for the model to generate a meaningful continuation
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,  # Enable sampling for more diverse outputs
        top_k=50,       # Top-k sampling
        top_p=0.95,     # Nucleus sampling
        temperature=0.1 # Controls randomness (lower values make outputs more deterministic)
    )

# Decode the generated tokens back to text
full_generated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Extract only the newly generated part (completion of the query)
input_length = len(tokenizer.decode(input_ids.input_ids[0], skip_special_tokens=True))
generated_completion = full_generated_text[input_length:].strip()

# Truncate the completion at the first semicolon (;)
if ';' in generated_completion:
    generated_completion = generated_completion.split(';')[0].strip() + ';'

print(f"Input: {test_input}")
print(f"Generated Completion: {generated_completion}")

Input: SELECT * FROM products JOIN 
Generated Completion: suppliers ON products.supplier_id = suppliers.supplier_id -> LEFT JOIN categories ON products.category_id = categories.category_id WHERE products.product_id < 15 AND products.product_id < 24;


In [19]:
generated_query = test_input + ' ' + generated_completion

norm1 = similarity.normalize_query(generated_query)
max_similarity = 0
most_similar_query = ""

for query2 in tqdm(sql_queries):
    norm2 = similarity.normalize_query(query2)  
    ast_sim = similarity.ast_similarity(norm1, norm2)
    component_sim = similarity.component_similarity(norm1, norm2)
    combined_score = (
        0.4 * ast_sim + 
        0.3 * component_sim.get("tables", 0) +
        0.2 * component_sim.get("columns", 0) +
        0.1 * component_sim.get("conditions", 0)
    )
    if combined_score > max_similarity:
        max_similarity = combined_score
        most_similar_query = query2

print("Generated query: \n", generated_query, '\n')
print("Most similar query: \n", most_similar_query, '\n')
print("Similarity of components: ", max_similarity)

100%|██████████| 5000/5000 [00:24<00:00, 200.88it/s]

Generated query: 
 SELECT * FROM products JOIN  suppliers ON products.supplier_id = suppliers.supplier_id -> LEFT JOIN categories ON products.category_id = categories.category_id WHERE products.product_id < 15 AND products.product_id < 24; 

Most similar query: 
 SELECT *
FROM products
 JOIN suppliers ON products.supplier_id = suppliers.supplier_id
 JOIN categories ON products.category_id = categories.category_id
WHERE products.supplier_id > 645 AND products.product_id < 913; 

Similarity of components:  0.7704892966360857
